In [99]:
# MTrend Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [34]:
bnf_1hr = pd.read_csv("../data/bnf_1hr_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2017)]
bnf_1hr.tail()

,open,high,low,close,datetime
21455,49942.4000,50032.1500,49835.2500,49939.2500,2024-06-13 11:15:00+05:30
21456,49940.7500,49984.8000,49921.5000,49967.4500,2024-06-13 12:15:00+05:30
21457,49965.8500,49975.5000,49860.7000,49902.9500,2024-06-13 13:15:00+05:30
21458,49902.9000,49905.7000,49799.6500,49821.0500,2024-06-13 14:15:00+05:30
21459,49818.8000,49920.6000,49817.9500,49889.3000,2024-06-13 15:15:00+05:30


In [191]:
spot_data = pd.read_csv("../data/bnf.csv")
spot_data["datetime"] = pd.to_datetime(spot_data["datetime"])
spot_data["datetime"] = spot_data["datetime"].dt.tz_localize(None)
spot_data = spot_data.drop(columns=["_id", "exchange", "tf", "instrument"])
spot_data = pl.DataFrame(spot_data)
# print(spot_data.head())
spot_data = spot_data.with_columns([pl.col("datetime").alias("index")])


def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label='left',
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                pl.col("v").sum().alias("v"),
            ]
        )
    )


bnf_resampled = resample(spot_data, "60m", pd.Timedelta(minutes=15))

bnf_df = bnf_resampled.to_pandas()
bnf_df = bnf_df.rename(
    columns={"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
)
bnf_1hr = bnf_df
bnf_1hr.head(10)

,datetime,open,high,low,close,volume
0,2017-01-02 09:15:00,18242.3000,18248.2000,17995.4000,18017.6000,0
1,2017-01-02 10:15:00,18017.1000,18035.7500,17874.1000,17903.9000,0
2,2017-01-02 11:15:00,17903.6000,17923.8500,17845.3500,17912.1000,0
3,2017-01-02 12:15:00,17911.6000,17951.1500,17893.6500,17946.9000,0
4,2017-01-02 13:15:00,17947.1500,18027.8000,17928.1000,18019.3000,0
5,2017-01-02 14:15:00,18018.8500,18052.8500,17945.5000,17962.7000,0
6,2017-01-02 15:15:00,17963.2500,17983.7500,17957.6000,17977.8000,0
7,2017-01-03 09:15:00,18002.7500,18069.4500,17831.7500,18066.6000,0
8,2017-01-03 10:15:00,18067.0000,18115.0000,18052.2500,18101.0500,0
9,2017-01-03 11:15:00,18100.2000,18106.8500,18044.4500,18079.1500,0


In [192]:
PORTFOLIO = 10_00_000
INDEX_LEVERAGE = 3
SLIPPAGE_FACTOR = 0.0001

In [193]:
def identify_pivots(df):
    # Initialize columns for bullish and bearish pivots
    df["Bullish Pivot"] = 0
    df["Bearish Pivot"] = 0

    # Identify bullish pivots
    for i in range(20, len(df)):
        if (df["low"][i - 20 : i - 10].min() >= df["low"][i - 10]) and (
            df["low"][i - 10 : i].min() >= df["low"][i - 10]
        ):
            df.at[i - 10, "Bullish Pivot"] = 1

    # Identify bearish pivots
    for i in range(20, len(df)):
        if (df["high"][i - 20 : i - 10].max() <= df["high"][i - 10]) and (
            df["high"][i - 10 : i].max() <= df["high"][i - 10]
        ):
            df.at[i - 10, "Bearish Pivot"] = 1

    return df


bnf_with_signals = identify_pivots(bnf_1hr)
bnf_with_signals.tail(50)

,datetime,open,high,low,close,volume,Bullish Pivot,Bearish Pivot
12792,2024-05-29 15:15:00,48518.4000,48624.9000,48460.7500,48563.0500,0,0,0
12793,2024-05-30 09:15:00,48313.6000,48919.9000,48313.6000,48654.0000,0,1,0
12794,2024-05-30 10:15:00,48649.3000,48788.6500,48632.1500,48646.1500,0,0,0
12795,2024-05-30 11:15:00,48642.9500,48713.2000,48531.0500,48644.1000,0,0,0
12796,2024-05-30 12:15:00,48644.9000,48874.2000,48614.7000,48851.1000,0,0,0
12797,2024-05-30 13:15:00,48850.2500,49044.6000,48766.1000,48852.4000,0,0,0
12798,2024-05-30 14:15:00,48850.5000,48905.4000,48545.0000,48637.2500,0,0,0
12799,2024-05-30 15:15:00,48642.2500,48838.6500,48642.2500,48822.8500,0,0,0
12800,2024-05-31 09:15:00,48895.1500,49079.0500,48719.5000,48719.5000,0,0,0
12801,2024-05-31 10:15:00,48717.8500,48821.2000,48612.4500,48698.5000,0,0,0


In [194]:
# bnf_1hr

In [195]:
def add_ma(df):
    df['MA'] = df['close'].rolling(20).mean()
    df['ABOVE MA'] = 0
    BULL_MASK = (
        df['close']>df['MA']
    )
    df.loc[BULL_MASK, "ABOVE MA"] = 1
    
    return df

bnf_1hr = add_ma(bnf_1hr)
# bnf_1hr

In [196]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample data creation
# df = pd.read_csv('your_data.csv')  # Load your data here

# Calculate the 50-period moving average (MA)
df['MA'] = df['close'].rolling(window=20).mean()

# Calculate the ABOVE MA column
df['ABOVE MA'] = (df['close'] > df['MA']).astype(int)

# Find where the close crosses above the MA
df['CROSS'] = (df['close'] > df['MA']) & (df['close'].shift(1) <= df['MA'].shift(1))

# Initialize list to store lengths of periods where close remains above MA
above_ma_periods = []

# Store the timestamps and period lengths for the histogram
cross_timestamps = []
cross_index = None
for i in range(1, len(df)):
    if df.loc[i, 'CROSS']:
        cross_index = i
    elif cross_index is not None and df.loc[i, 'close'] < df.loc[i, 'MA']:
        period_length = i - cross_index
        above_ma_periods.append(period_length)
        cross_timestamps.append(df.index[cross_index])
        cross_index = None

# Calculate the average number of candles above MA after crossing
average_above_ma = sum(above_ma_periods) / len(above_ma_periods) if above_ma_periods else 0

print(f'Average number of candles above MA after crossing: {average_above_ma}')


Average number of candles above MA after crossing: 10.183459277917716


In [197]:
df

,datetime,open,high,low,close,volume,Bullish Pivot,Bearish Pivot,MA,ABOVE MA,CROSS
0,2017-01-02 09:15:00,18242.3000,18248.2000,18052.1500,18052.1500,0,0,0,NaN,0,False
1,2017-01-02 09:30:00,18051.2500,18052.3500,17995.4000,18049.8000,0,0,0,NaN,0,False
2,2017-01-02 09:45:00,18048.2500,18048.5500,18003.6000,18020.6000,0,0,0,NaN,0,False
3,2017-01-02 10:00:00,18019.6500,18040.9500,18007.9000,18017.6000,0,0,0,NaN,0,False
4,2017-01-02 10:15:00,18017.1000,18035.7500,18014.4500,18014.7000,0,0,0,NaN,0,False
...,...,...,...,...,...,...,...,...,...,...,...
45843,2024-06-07 14:15:00,49817.3000,49909.2000,49761.5000,49818.7000,0,0,0,49700.5250,1,False
45844,2024-06-07 14:30:00,49819.0000,49849.8500,49711.2500,49735.4500,0,0,0,49717.5725,1,False
45845,2024-06-07 14:45:00,49734.0500,49887.6000,49734.0500,49847.3500,0,0,0,49736.3650,1,False
45846,2024-06-07 15:00:00,49838.2500,49866.9000,49744.2000,49777.7500,0,0,0,49754.7575,1,False


In [198]:
df = bnf_1hr

df['MA'] = df['close'].rolling(window=20).mean()

# Calculate the ABOVE MA column
df['ABOVE MA'] = (df['close'] > df['MA']).astype(int)

# Find where the close crosses above the MA
df['CROSS'] = (df['close'] > df['MA']) & (df['close'].shift(1) <= df['MA'].shift(1))

# Initialize list to store lengths of periods where close remains above MA
above_ma_periods = []

# Store the timestamps and period lengths for the histogram
cross_timestamps = []
cross_index = None
cross_time = None
for i in range(1, len(df)):
    if df.loc[i, 'CROSS']:
        cross_index = i
        cross_time = df.loc[i, 'datetime']
    elif cross_index is not None and df.loc[i, 'close'] < df.loc[i, 'MA']:
        period_length = i - cross_index
        print(cross_time ,period_length)
        above_ma_periods.append(period_length)
        cross_timestamps.append(cross_time)
        cross_index = None
    elif cross_index is not None and i==len(df)-1:
        period_length = i - cross_index
        print(cross_time ,period_length)
        above_ma_periods.append(period_length)
        cross_timestamps.append(cross_time)
        cross_index = None

# Calculate the average number of candles above MA after crossing
average_above_ma = sum(above_ma_periods) / len(above_ma_periods) if above_ma_periods else 0

print(f'Average number of candles above MA after crossing: {average_above_ma}')

# Prepare the data for plotting
cross_data = pd.DataFrame({'Timestamp': cross_timestamps, 'Candles Above MA': above_ma_periods})

# Plot the histogram
# plt.figure(figsize=(12, 6))
# plt.bar(cross_data['Timestamp'], cross_data['Candles Above MA'], width=1)
# plt.xlabel('Timestamp')
# plt.ylabel('Number of Candles Above 50MA')
# plt.title('Number of Candles Above 50MA After Each Cross')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


2017-01-05 09:15:00 70
2017-01-19 11:15:00 2
2017-01-24 09:15:00 28
2017-02-01 10:15:00 1
2017-02-01 12:15:00 31
2017-02-10 09:15:00 5
2017-02-10 15:15:00 3
2017-02-13 13:15:00 5
2017-02-14 14:15:00 4
2017-02-16 11:15:00 40
2017-03-01 09:15:00 12
2017-03-06 09:15:00 3
2017-03-06 14:15:00 10
2017-03-08 13:15:00 14
2017-03-10 14:15:00 23
2017-03-24 09:15:00 40
2017-04-03 09:15:00 1
2017-04-03 11:15:00 19
2017-04-11 09:15:00 19
2017-04-18 09:15:00 5
2017-04-21 14:15:00 32
2017-04-28 12:15:00 12
2017-05-03 13:15:00 1
2017-05-04 09:15:00 21
2017-05-09 10:15:00 1
2017-05-09 12:15:00 2
2017-05-10 09:15:00 13
2017-05-15 09:15:00 3
2017-05-15 13:15:00 3
2017-05-16 10:15:00 13
2017-05-19 09:15:00 1
2017-05-19 15:15:00 1
2017-05-22 10:15:00 2
2017-05-25 09:15:00 20
2017-05-30 09:15:00 3
2017-05-30 13:15:00 12
2017-06-01 13:15:00 20
2017-06-06 13:15:00 1
2017-06-07 09:15:00 21
2017-06-13 10:15:00 3
2017-06-16 09:15:00 2
2017-06-16 12:15:00 21
2017-06-21 13:15:00 9
2017-06-29 09:15:00 3
2017-06-30 

In [199]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.2,
    horizontal_spacing=0.2,
)

fig.add_trace(
    go.Bar(
        x=cross_data["Timestamp"],
        y=cross_data["Candles Above MA"],
        marker_color='white',
        width=1,
    ),
    row=1,
    col=1,
)


# Updating layout
fig.update_layout(
    height=600,
    width=1600,
    title_text="Number of Candles Above 50MA After Each Cross",
    xaxis_title="Timestamp",
    yaxis_title="Number of Candles Above 50MA",
    plot_bgcolor='black',
    paper_bgcolor='white',
)
fig.update_xaxes(rangeslider_visible=True)
fig.update_yaxes(dtick=10)

app = Dash(__name__)
app.layout = html.Div([dcc.Graph(figure=fig)])

if __name__ == '__main__':
    app.run_server(debug=True)